# NPM Kompleks Ağ Analizi (Top N)

Bu defter, popüler Top N NPM paketini yönlü bir ağ (Dependent → Dependency) olarak modelleyip, in-degree / out-degree / betweenness merkeziyet metrikleriyle yapısal riski inceler.

**Amaç**
- Bağımlılık ağındaki konuma dayalı kritik düğümleri belirlemek
- Zincirleme etki (cascading impact) potansiyelini nicel görmek
- Sonuçları yineleyebilir şekilde görselleştirmek ve raporlamak

**Yöntem**
- Veri: Liste her çalıştırmada API’lerden çekilir (ecosyste.ms / npm registry / npms.io yedekli).
- Ağ Modeli: NetworkX DiGraph; kenar yönü Dependent → Dependency (bağımlı paket → bağımlılık).
- Metrikler: In-Degree (gelen), Out-Degree (giden), Betweenness (köprü rol).
- Performans: Büyük graf’larda betweenness yaklaşık örnekleme ile hızlandırılır.
- Çıktılar: Tüm sonuçlar `results/` klasörüne kaydedilir (CSV/MD/JSON + PNG/SVG).

Varsayılan Top N = 1000 (değiştirilebilir).

## 1) İçe Aktarım ve Parametreler

Yardımcı fonksiyonlar `analysis_helpers.py` içindedir. Varsayılan `OUTDIR` = `results/`.

In [ ]:
# İçe aktarımlar ve temel parametreler
from pathlib import Path
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
from analysis_helpers import (
fetch_top_packages,
build_dependency_graph,
compute_metrics,
save_edges,
save_metrics,
save_report,
    compute_risk_scores,
    save_risk_scores,
    robustness_remove_and_stats,
)

import json
import numpy as np

TOP_N = 1000  # Top N paket (varsayılan)
INCLUDE_PEER_DEPS = False  # peerDependencies'i eklemek icin True yapin
RISK_WEIGHTS = (0.5, 0.2, 0.3)  # (w_in, w_out, w_btw)
SAMPLE_K = None  # Betweenness örnekleme (None=tahmine dayalı)
OUTDIR = Path('results')
OUTDIR.mkdir(parents=True, exist_ok=True)
OUTDIR


## 2) Top N Paketleri Yükle

Liste, indirilmeye göre sıralı şekilde API’lerden çekilir. Öncelik ecosyste.ms; gerekirse npm registry / npms.io yedekleri kullanılır. Bu yaklaşım sabit dosyaya bağımlılığı kaldırır ve güncel popülerliği yansıtır.

In [ ]:
top_packages = fetch_top_packages(TOP_N)
print(f'API’lerden {len(top_packages)} paket çekildi')
len(top_packages), top_packages[:20]

## 3) Yönlü Ağı Kur

Kenar yönü Dependent → Dependency (paket → kullandığı bağımlılık). Bu yön, bir paketin ele geçirilmesi halinde etki akışını (bağımlıdan bağımlılığa) analiz etmeyi kolaylaştırır.

In [ ]:
G, top_set = build_dependency_graph(top_packages, cache_path=OUTDIR / 'cache_deps.json', include_peer_deps=INCLUDE_PEER_DEPS)
G.number_of_nodes(), G.number_of_edges()

> Çıktı Notu: Not: Ağ kuruldu; düğüm/kenar sayıları üstte raporlandı (G.number_of_nodes/edges).

## 4) Merkeziyet Metrikleri

- In-Degree: Düğüme gelen kenar (bu pakete dayanan paket sayısı) → ele geçirilirse etki alanı.
- Out-Degree: Düğümün dış bağımlılık sayısı → bağımlılık zinciri uzunluğu/karmaşıklığı.
- Betweenness: En kısa yollardaki aracılık → köprü/tek hata noktası riski.

Not: Büyük graf’larda betweenness örneklemeli (k) hesaplanır; notebook ilk hücrede `SAMPLE_K` ile ayarlanabilir.

In [ ]:
in_deg, out_deg, btw = compute_metrics(G, sample_k=SAMPLE_K)
nodes = G.number_of_nodes(); edges = G.number_of_edges()
nodes, edges

> Çıktı Notu: Not: Merkeziyet metrikleri (in/out/between) hesaplandı; sonraki adımlar bu metriklere dayanır.

## 5) Liderler: İlk 10 (Özet + Görselleştirme)
In/Out-Degree ve Betweenness için ilk 10 düğümü hem metin hem grafik olarak sunar. Bu listeler ağın yapısal omurgasını ve kritik köprü düğümlerini öne çıkarır.

In [ ]:
top_in_all = sorted(in_deg.items(), key=lambda kv: kv[1], reverse=True)[:10]
top_btw_all = sorted(btw.items(), key=lambda kv: kv[1], reverse=True)[:10]
top_in_top = sorted(((n, in_deg.get(n, 0)) for n in top_set), key=lambda kv: kv[1], reverse=True)[:10]
top_btw_top = sorted(((n, btw.get(n, 0.0)) for n in top_set), key=lambda kv: kv[1], reverse=True)[:10]
top_in_all, top_btw_all, top_in_top, top_btw_top

## 6) Sonuçları Kaydet
Kenarlıklar, merkeziyet metrikleri ve kısa rapor dosyaya yazılır. Böylece defter dışı analiz ve kıyaslamalara zemin hazırlanır.

In [ ]:
save_edges(G, OUTDIR / 'edges.csv')
save_metrics(in_deg, out_deg, btw, top_set, OUTDIR / 'metrics.csv')
save_report(in_deg, out_deg, btw, top_set, OUTDIR / 'report.md')
(OUTDIR / 'top_packages.txt').write_text(os.linesep.join(top_packages), encoding='utf-8')
sorted(OUTDIR.iterdir())

## 7) Hızlı Doğrulama
Rastgele paketler için registry bağımlılıkları ile graf kenarlarını karşılaştırır. Küçük farklar sürüm farklılıkları ve metadata gecikmelerinden kaynaklanabilir.

In [ ]:
ENABLE_VALIDATION = False
if ENABLE_VALIDATION:
    import random, requests, urllib.parse
    def get_registry_deps(name: str):
        url = f'https://registry.npmjs.org/{urllib.parse.quote(name, safe="")}'
        r = requests.get(url, timeout=30)
        if r.status_code != 200:
            return {}
        data = r.json()
        latest = (data.get('dist-tags') or {}).get('latest')
        versions = data.get('versions') or {}
        obj = versions.get(latest) if latest in versions else {}
        return obj.get('dependencies') or {}
    
    sample = random.sample(top_packages, k=min(5, len(top_packages)))
    mismatches = {}
    for pkg in sample:
        deps_live = set(get_registry_deps(pkg).keys())
        deps_graph = set(v for u,v in G.out_edges(pkg))
        if deps_live != deps_graph:
            mismatches[pkg] = {
                'live_only': sorted(list(deps_live - deps_graph)),
                'graph_only': sorted(list(deps_graph - deps_live)),
            }
    mismatches
else:
    print('Doğrulama atlandı (ENABLE_VALIDATION=False)')


## 8) Tüm Ağ Çizimi (Top N + Bağımlılıklar)
Top N düğümler turuncu, diğerleri mavi; düğüm boyutu in-degree ile ölçeklenir. Ağın genel topolojisi ve yıldız/omurga yapıları gözlemlenir.

In [ ]:
import math
N = G.number_of_nodes()
pos = nx.spring_layout(G, k=1/math.sqrt(max(N,1)), seed=42)
max_in = max(in_deg.values()) if in_deg else 1
node_sizes = [100 + (400 * (in_deg.get(n,0) / max_in if max_in else 0)) for n in G.nodes()]
node_colors = ['tab:orange' if n in top_set else 'tab:blue' for n in G.nodes()]
plt.figure(figsize=(12, 9), dpi=150)
nx.draw_networkx_edges(G, pos, arrows=False, width=0.5, alpha=0.25)
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.85, linewidths=0.2, edgecolors='#444444')
plt.title('NPM Top N Bağımlılık Ağı (Tümü)')
plt.axis('off')
out_png = OUTDIR / 'network_full_topN.png'
out_svg = OUTDIR / 'network_full_topN.svg'
plt.tight_layout(); plt.savefig(out_png); plt.savefig(out_svg)
(out_png, out_svg)


## 9) Sadece Top N (İndüklenmiş Alt-Ağ)
Sadece Top N düğümlerin oluşturduğu alt-ağ; etiketler az düğüm olduğunda gösterilir. Kohort içi yapısal ilişkiler netleşir.

In [ ]:
H = G.subgraph(top_set).copy()
HN = H.number_of_nodes() or 1
posH = nx.spring_layout(H, k=1/math.sqrt(HN), seed=42)
max_in_H = max((in_deg.get(n,0) for n in H.nodes()), default=1)
node_sizes_H = [120 + (480 * (in_deg.get(n,0) / max_in_H if max_in_H else 0)) for n in H.nodes()]
node_colors_H = ['tab:orange' for _ in H.nodes()]
plt.figure(figsize=(10, 8), dpi=150)
nx.draw_networkx_edges(H, posH, arrows=False, width=0.6, alpha=0.35)
nx.draw_networkx_nodes(H, posH, node_size=node_sizes_H, node_color=node_colors_H, alpha=0.9, linewidths=0.2, edgecolors='#444444')
if H.number_of_nodes() <= 60:
    nx.draw_networkx_labels(H, posH, font_size=6)
plt.title('NPM Top N Bağımlılık Ağı (Sadece Top N)')
plt.axis('off')
out_png2 = OUTDIR / 'network_topN_only.png'
out_svg2 = OUTDIR / 'network_topN_only.svg'
plt.tight_layout(); plt.savefig(out_png2); plt.savefig(out_svg2)
(out_png2, out_svg2)


## 14) Varsayımlar ve Sınırlamalar
- Kenar yönü Dependent → Dependency; yayılım analizi için uygundur.
- Varsayılan olarak yalnız `dependencies` dahil; `peerDependencies` isteğe bağlı eklenebilir.
- Global dependent sayıları dahil değildir; ecosyste.ms verisi eklenebilir.
- En güncel sürüm kullanılır; eski sürümlerde bağımlılıklar farklı olabilir.

In [ ]:
# In-Degree ilk 10 (tüm düğümler)
top10_in = sorted(in_deg.items(), key=lambda kv: kv[1], reverse=True)[:10]
names_in = [n for n,_ in top10_in][::-1]
vals_in = [v for _,v in top10_in][::-1]

# Out-Degree ilk 10 (tüm düğümler)
top10_out = sorted(out_deg.items(), key=lambda kv: kv[1], reverse=True)[:10]
names_out = [n for n,_ in top10_out][::-1]
vals_out = [v for _,v in top10_out][::-1]

# Betweenness ilk 10 (tüm düğümler)
top10_btw = sorted(btw.items(), key=lambda kv: kv[1], reverse=True)[:10]
names_btw = [n for n,_ in top10_btw][::-1]
vals_btw = [v for _,v in top10_btw][::-1]

fig, axes = plt.subplots(1, 3, figsize=(20, 6), dpi=150)
# In-Degree grafiği
axes[0].barh(names_in, vals_in, color='tab:blue')
axes[0].set_title('In-Degree İlk 10')
axes[0].set_xlabel('In-Degree')
axes[0].tick_params(axis='y', labelsize=8)

# Out-Degree grafiği
axes[1].barh(names_out, vals_out, color='tab:green')
axes[1].set_title('Out-Degree İlk 10')
axes[1].set_xlabel('Out-Degree')
axes[1].tick_params(axis='y', labelsize=8)

# Betweenness grafiği
axes[2].barh(names_btw, vals_btw, color='tab:orange')
axes[2].set_title('Betweenness İlk 10')
axes[2].set_xlabel('Betweenness')
axes[2].tick_params(axis='y', labelsize=8)

plt.tight_layout()
out_combo = OUTDIR / 'top10_leaders.png'
out_combo_svg = OUTDIR / 'top10_leaders.svg'
plt.savefig(out_combo)
plt.savefig(out_combo_svg)
print(out_combo)

# Ayrı ayrı da kaydedelim
plt.figure(figsize=(8,6), dpi=150); plt.barh(names_in, vals_in, color='tab:blue'); plt.title('In-Degree İlk 10'); plt.xlabel('In-Degree'); plt.tight_layout(); plt.savefig(OUTDIR / 'top10_in_degree.png')
plt.savefig(OUTDIR / 'top10_in_degree.svg')
plt.figure(figsize=(8,6), dpi=150); plt.barh(names_out, vals_out, color='tab:green'); plt.title('Out-Degree İlk 10'); plt.xlabel('Out-Degree'); plt.tight_layout(); plt.savefig(OUTDIR / 'top10_out_degree.png')
plt.savefig(OUTDIR / 'top10_out_degree.svg')
plt.figure(figsize=(8,6), dpi=150); plt.barh(names_btw, vals_btw, color='tab:orange'); plt.title('Betweenness İlk 10'); plt.xlabel('Betweenness'); plt.tight_layout(); plt.savefig(OUTDIR / 'top10_betweenness.png')
plt.savefig(OUTDIR / 'top10_betweenness.svg')


## 10) Derece Dağılımları (Histogram)
In-degree ve out-degree dağılımları (log ölçek). Kuyruk davranışı ve yoğunlaşmalar görülür.

In [ ]:
indeg_vals = np.array(list(in_deg.values()))
outdeg_vals = np.array(list(out_deg.values()))
fig, ax = plt.subplots(1,2, figsize=(12,4), dpi=150)
ax[0].hist(indeg_vals, bins=30, color='tab:blue', alpha=0.8)
ax[0].set_title('In-Degree Dağılımı'); ax[0].set_yscale('log')
ax[0].set_xlabel('In-Degree'); ax[0].set_ylabel('Frekans')
ax[1].hist(outdeg_vals, bins=30, color='tab:green', alpha=0.8)
ax[1].set_title('Out-Degree Dağılımı'); ax[1].set_yscale('log')
ax[1].set_xlabel('Out-Degree'); ax[1].set_ylabel('Frekans')
plt.tight_layout()
plt.savefig(OUTDIR / 'degree_histograms.png')
plt.savefig(OUTDIR / 'degree_histograms.svg')


> Çıktı Notu: In/Out-degree histogramları kaydedildi (degree_histograms.*).

## 11) Korelasyonlar (Dağılım Grafikleri)
In-Degree vs Betweenness ve In-Degree vs Out-Degree ilişkileri; köprü düğümlerin derecesi ve dış bağımlılıklarının etkisi.

In [ ]:
nodes_list = list(G.nodes())
x_in = np.array([in_deg.get(n,0) for n in nodes_list])
y_btw = np.array([btw.get(n,0.0) for n in nodes_list])
y_out = np.array([out_deg.get(n,0) for n in nodes_list])
fig, ax = plt.subplots(1,2, figsize=(12,4), dpi=150)
ax[0].scatter(x_in, y_btw, s=10, alpha=0.6, color='tab:orange')
ax[0].set_title('In-Degree vs Betweenness')
ax[0].set_xlabel('In-Degree'); ax[0].set_ylabel('Betweenness')
ax[1].scatter(x_in, y_out, s=10, alpha=0.6, color='tab:green')
ax[1].set_title('In-Degree vs Out-Degree')
ax[1].set_xlabel('In-Degree'); ax[1].set_ylabel('Out-Degree')
plt.tight_layout()
plt.savefig(OUTDIR / 'scatter_correlations.png')
plt.savefig(OUTDIR / 'scatter_correlations.svg')


> Çıktı Notu: Korelasyon saçılımları kaydedildi (scatter_correlations.*).

## 12) Bağlanırlık ve Bileşenler
Zayıf bağlanırlık bileşen sayısı, en büyük bileşen boyutu ve ortalama dereceler. Ağın parçalanabilirliği hakkında sezgi verir.

In [ ]:
W = G.to_undirected()
components = list(nx.connected_components(W))
components_sizes = sorted([len(c) for c in components], reverse=True)
stats = {
    'nodes': G.number_of_nodes(),
    'edges': G.number_of_edges(),
    'components_count': len(components),
    'largest_component_size': components_sizes[0] if components_sizes else 0,
    'avg_in_degree': (sum(in_deg.values())/len(in_deg)) if in_deg else 0,
    'avg_out_degree': (sum(out_deg.values())/len(out_deg)) if out_deg else 0,
}
(OUTDIR / 'graph_stats.json').write_text(json.dumps(stats, ensure_ascii=False, indent=2), encoding='utf-8')
stats

## 13) Köprü Kenarlar (Edge Betweenness)
Edge betweenness’a göre en kritik 10 kenar. Bu kenarların kopması, alt-ağların ayrışmasına yol açabilir.

In [ ]:
import csv
eb = nx.edge_betweenness_centrality(G, normalized=True)
top_edges = sorted(eb.items(), key=lambda kv: kv[1], reverse=True)[:10]
with (OUTDIR / 'edge_betweenness_top10.csv').open('w', newline='', encoding='utf-8') as f:
    w = csv.writer(f); w.writerow(['u','v','edge_betweenness'])
    [w.writerow([u,v, f'{val:.6f}']) for (u,v), val in top_edges]
top_edges

## 13) Risk Skoru (Bileşik)
Normalize in/out/between ile ağırlıklı (w_in,w_out,w_btw) risk skoru. İlk 20’yi dışa aktaralım ve görselleştirelim.

In [ ]:
risk = compute_risk_scores(in_deg, out_deg, btw, *RISK_WEIGHTS)
save_risk_scores(risk, in_deg, out_deg, btw, top_set, OUTDIR / 'risk_scores.csv')
# İlk 20 risk liderini görselleştir (PNG+SVG)
top20 = sorted(risk.items(), key=lambda kv: kv[1], reverse=True)[:20]
names = [n for n,_ in top20][::-1]; vals=[v for _,v in top20][::-1]
plt.figure(figsize=(8,8), dpi=150); plt.barh(names, vals, color='tab:red'); plt.title('Risk Skoru İlk 20'); plt.xlabel('Risk'); plt.tight_layout();
plt.savefig(OUTDIR / 'top20_risk.png'); plt.savefig(OUTDIR / 'top20_risk.svg')
(len(risk), len(top20))

> Çıktı Notu: Risk skorları kaydedildi (risk_scores.csv); Top 20 görseller üretildi.

## 14) Robustluk Analizi (Risk Tabanlı Kaldırma)
Risk skoruna göre ilk 1/3/5 düğümü kaldırınca bağlanırlık nasıl değişiyor?

In [ ]:
from analysis_helpers import robustness_remove_and_stats
ranked = [n for n,_ in sorted(risk.items(), key=lambda kv: kv[1], reverse=True)]
ks = [1,3,5]
results = {}
for k in ks:
    removed = ranked[:k]
    stats = robustness_remove_and_stats(G, removed)
    results[str(k)] = {"removed": removed, "stats": stats}
(OUTDIR / 'robustness_risk.json').write_text(json.dumps(results, ensure_ascii=False, indent=2), encoding='utf-8')
results

> Çıktı Notu: Robustluk sonuçları kaydedildi (robustness_risk.json).

## 15) Basamaklanma (Cascading Impact)

Ağ yönü Dependent → Dependency olduğu için, bir bağımlılığın ele geçirilmesi halinde etkilenen paketler, bu düğüme ulaşabilen (dependents) düğümlerdir. Bu analizde risk liderleri için ters yönde erişilebilen düğüm sayısını hesaplıyoruz.

In [ ]:
from analysis_helpers import cascade_impact_counts, save_cascade_impact
seeds = [n for n,_ in sorted(risk.items(), key=lambda kv: kv[1], reverse=True)[:20]]
impact = cascade_impact_counts(G, seeds)
save_cascade_impact(impact, OUTDIR / 'cascade_impact_top20.csv')
pairs = sorted(impact.items(), key=lambda kv: kv[1], reverse=True)[:20]
names = [n for n,_ in pairs][::-1]; vals=[v for _,v in pairs][::-1]
plt.figure(figsize=(8,8), dpi=150); plt.barh(names, vals, color='tab:purple'); plt.title('Basamaklanma Etkisi (Top 20 Risk Lideri)'); plt.xlabel('Etkilenen paket sayısı (ters yön ulaşılan)'); plt.tight_layout();
plt.savefig(OUTDIR/'cascade_impact_top20.png'); plt.savefig(OUTDIR/'cascade_impact_top20.svg')


> Çıktı Notu: cascade_impact_top20.csv ve görseller üretildi; bu metrik, ele geçirilen bir paketin potansiyel etki alanını (dependents sayısı) yaklaşık olarak gösterir.

In [ ]:
# Risk vs Basamaklanma (destekleyici korelasyon)
rv = [risk.get(n,0.0) for n,_ in pairs]; cv = [v for _,v in pairs]
plt.figure(figsize=(5,4), dpi=150); plt.scatter(cv, rv, alpha=0.7, color='tab:orange'); plt.xlabel('Basamaklanma (impacted_count)'); plt.ylabel('Risk Skoru'); plt.title('Risk vs Basamaklanma'); plt.tight_layout();
plt.savefig(OUTDIR/'risk_vs_cascade.png'); plt.savefig(OUTDIR/'risk_vs_cascade.svg')


> Çıktı Notu: risk_vs_cascade görselleri ile risk skorunun basamaklanma etkisiyle ilişkisi görselleştirildi.